In [1]:
import ee
ee.Initialize()

In [2]:
import sys
sys.path.append('/home/sam/git/github/gee-atmcorr-S2-batch/bin')
from sixs_emulator_ee_sentinel2_batch import SixS_emulator
from atmcorr_input import Atmcorr_input
from atmospheric_correction import atmospheric_correction
from radiance import radiance_from_TOA

In [3]:
# a place and a mission
geom = ee.Geometry.Point(-157.816222, 21.297481)
mission = 'COPERNICUS/S2'  

In [4]:
# image collection
ic = ee.ImageCollection(mission)\
    .filterBounds(geom)\
    .filterDate('2017-01-01','2017-06-12')\
    .filter(ee.Filter.lt('MEAN_SOLAR_ZENITH_ANGLE',75))

In [5]:
# 6S emulator
se = SixS_emulator(mission)

In [6]:
# load interpolate look up tables
se.load_iLUTs('/home/sam/git/github/gee-atmcorr-S2-batch/files/iLUTs/S2A_MSI/Continental/view_zenith_0/')

In [7]:
# extract atmcorr inputs as feature collection
Atmcorr_input.geom = geom  # specify target location (would use image centroid otherwise)
atmcorr_inputs = ic.map(Atmcorr_input.extractor).getInfo()
features = atmcorr_inputs['features']

In [8]:
# atmospherically correct image collection

ic_atmospherically_corrected = []

for feature in features:

    # at-sensor radiance 
    toa = ee.Image(mission+'/'+feature['properties']['imgID'])
    rad = radiance_from_TOA(toa, feature)

    # 6S emulator
    cc = se.run(feature['properties']['atmcorr_inputs'])

    # atmospheric correction
    SR = atmospheric_correction(rad, cc)
    ic_atmospherically_corrected.append(SR)

In [9]:
# Earth Engine image collection
ic_atmospherically_corrected = ee.ImageCollection(ic_atmospherically_corrected)

In [10]:
# test
SR = ee.Image(ic_atmospherically_corrected.first())
print(SR.reduceRegion(ee.Reducer.mean(),geom).getInfo())

{'B2': 0.21544715254774705, 'B5': 0.267519227838868, 'B7': 0.302642586072869, 'B10': 0.4969368640425421, 'B6': 0.30273065960250845, 'B11': 0.2493073375084754, 'B9': 0.4176718790944925, 'B8A': 0.29977697553848054, 'B3': 0.23786308414737306, 'B8': 0.3079101947591835, 'B4': 0.26133856162825925, 'B12': 0.187208378759739, 'B1': 0.13939643278561717}


In [12]:
SR = ee.Image(ic_atmospherically_corrected.first())

from IPython.display import display, Image
region = geom.buffer(5000).bounds().getInfo()['coordinates']
image = Image(url=SR.select(['B4','B3','B2']).getThumbUrl({
                'region':region,
                'min':0,
                'max':0.3
                }))
display(image)